In [1]:
#GLOBALS
import numpy as np
import general_robotics_toolbox as rox #ik
import time
import math
from Arm_Lib import Arm_Device #

# Create robot arm object
Arm = Arm_Device()
time.sleep(.1) #tenth of a second delay

# print(0 if type(Arm.Arm_serial_servo_read(1)) == class(None) else int(Arm.Arm_serial_servo_read(1)))
# print(type(Arm.Arm_serial_servo_read(1)))

# print(0 if Arm.Arm_serial_servo_read(1) == None else Arm.Arm_serial_servo_read(1))
# print(Arm.Arm_serial_servo_read(1) if Arm.Arm_serial_servo_read(1) != None else 0)

#Defining variables
angle = 90
s_time = 1000 #milliseconds
gripper = 6

# Define the home configuration of the robot
H0 = np.eye(4)

print("passed init")

passed init


In [2]:
def home(): #set arm to home position after task is complete. Useful for ik
    Arm.Arm_serial_servo_write6(angle, angle, angle, angle, angle, angle-85, s_time) #FIXME
home()
print("passed home")

passed home


In [3]:
################################################################################################################################################################################################################################################################
# #IK Function 1
################################################################################################################################################################################################################################################################

# import the needed modules (as is used to rename the module for easier referencing)
import math
import numpy as np
import general_robotics_toolbox as rox

# define the Jacobian inverse 
def jacobian_inverse(robot,q0,Rd,Pd,Nmax,alpha,tol):
# the inputs are 
	# robot: of the Robot class defined in rox. contains rotation axes, position vectors, joint types
	# q0: the initial guess joint angles as a 5x1 numpy array
	# Rd: the desired 3x3 rotation matrix (R_{0T}) as a numpy array
	# Pd: the desired 3x1 position vector (P_{0T}) as a numpy array
	# Nmax: the maximum number of allowed iterations
	# alpha: the update parameter
	# tol: the tolerances for [roll, pitch, yaw, x, y, z] as a 6x1 numpy array

# set up storage space
    n = len(q0)
    q = np.zeros((n,Nmax+1))
    q[:,0] = q0
    p0T = np.zeros((3,Nmax+1))
    RPY0T = np.zeros((3,Nmax+1))
    iternum = 0

# compute the forward kinematics
    H = rox.fwdkin(robot,q[:,0])
    R = H.R
    P = H.p
    P = np.array([[P[0]], [P[1]], [P[2]]])

# get the initial error
    dR = np.matmul(R, np.transpose(Rd))
    r = np.array(rox.R2rpy(dR))[None]
    dX = np.concatenate((np.transpose(r), P-Pd))

# iterate while any error element is greater than its tolerance
    while (np.absolute(dX) > tol).any():
	# stop execution if the maximum number of iterations is exceeded
        if iternum < Nmax:
		# compute the forward kinematics
            H = rox.fwdkin(robot, q[:,iternum])
            R = H.R
            p0T = H.p
            p0T = np.array([[p0T[0]], [p0T[1]], [p0T[2]]])
		# compute the error
            dR = np.matmul(R , np.transpose(Rd))
            r = np.array(rox.R2rpy(dR))[None]
            dX = np.concatenate((np.transpose(r), p0T-Pd))

		# calculate the Jacobian matrix
            Jq = rox.robotjacobian(robot, q[:, iternum])
		# compute the update
            j = np.matmul(np.linalg.pinv(Jq), dX)
		# use the update to generate a new q
            q[:, iternum+1] = q[:, iternum] - np.transpose((alpha * j))
            iternum = iternum + 1
        else:
            break
	# return the final estimate of q
    return q[:, iternum]

def main():

	#define inputs
	
    #qd=[30;45;80;25;40]
    #Rd = np.array([ [-0.75, -0.1047, -0.6531], [-0.433, 0.8241, 0.3652], [0.5, 0.5567, -0.6634]])
    #Pd = np.array([[0.2058], [0.1188], [0.1464]])
    #q0 = np.array(np.transpose([25, 50, 75, 30, 30]))*math.pi/180

    #qd=[0;45;135;45;135]
    Rd = np.array([ [-0.7071, -0.5, -0.5], [0, -0.7071, 0.7071], [-0.7071, 0.5, 0.5]])
    Pd = np.array([[0.1494], [0], [0.3367]])
	# convert initial guess to radians
    q0 = np.array(np.transpose([-5, 50, 130, 50, 130]))*math.pi/180
    
    tol = np.array(np.transpose([[0.02, 0.02, 0.02, 0.001, 0.001, 0.001]]))
    Nmax = 200
    alpha = 0.1

    # Define all the joint lengths [m]
    l0 = 61 * 10**-3
    l1 = 43.5 * 10**-3
    l2 = 82.85 * 10**-3
    l3 = 82.85 * 10**-3
    l4 = 73.85 * 10**-3
    l5 = 54.57 * 10**-3

	# define the unit vectors
    ex = np.array([1, 0, 0])
    ey = np.array([0, 1, 0])
    ez = np.array([0, 0, 1])

    # Define the position vectors from i-1 -> i
    P01 = (l0 + l1) * ez
    P12 = np.zeros(3)
    P23 = l2 * ex
    P34 = -1*l3 * ez
    P45 = np.zeros(3)
    P5T = -1*(l4 + l5) * ex

	# define the class inputs: rotation axes (H), position vectors (P), and joint_type
    H = np.array([ez, -1*ey, -1*ey, -1*ey, -1*ex]).T
    P = np.array([P01, P12, P23, P34, P45, P5T]).T
    joint_type = [0,0,0,0,0]

	# define the Robot class
    robot = rox.Robot(H, P, joint_type)
	
	# compute the inverse kinematics
    q = jacobian_inverse(robot,q0,Rd,Pd,Nmax,alpha,tol)
	# convert solution to degrees
    q = q * 180 / math.pi
    print(q) #this gives an array of q values for the end effector. Could be simply inputted to servo commands like we used for fk. Arm.Arm_serial_servo_write6(angle, angle, angle, angle, angle, angle-90, s_time) #FIXME
    Arm.Arm_serial_servo_write6(q[0], q[1], q[2], q[3], q[4], 90, s_time) #FIXME
    
if __name__ == "__main__" :
    main()
    
print("passed IK")


[ -0.28022737  45.25692223 134.83454941  45.17431152 134.67077778]
passed IK


In [115]:
################################################################################################################################################################################################################################################################
# #IK Function 1
################################################################################################################################################################################################################################################################

# import the needed modules (as is used to rename the module for easier referencing)
import math
import numpy as np
import general_robotics_toolbox as rox

# define the Jacobian inverse 
def jacobian_inverse(robot,q0,Rd,Pd,Nmax,alpha,tol):
# the inputs are 
	# robot: of the Robot class defined in rox. contains rotation axes, position vectors, joint types
	# q0: the initial guess joint angles as a 5x1 numpy array
	# Rd: the desired 3x3 rotation matrix (R_{0T}) as a numpy array
	# Pd: the desired 3x1 position vector (P_{0T}) as a numpy array
	# Nmax: the maximum number of allowed iterations
	# alpha: the update parameter
	# tol: the tolerances for [roll, pitch, yaw, x, y, z] as a 6x1 numpy array

# set up storage space
    n = len(q0)
    q = np.zeros((n,Nmax+1))
    q[:,0] = q0
    p0T = np.zeros((3,Nmax+1))
    RPY0T = np.zeros((3,Nmax+1))
    iternum = 0

# compute the forward kinematics
    H = rox.fwdkin(robot,q[:,0])
    R = H.R
    P = H.p
    P = np.array([[P[0]], [P[1]], [P[2]]])

# get the initial error
    dR = np.matmul(R, np.transpose(Rd))
    r = np.array(rox.R2rpy(dR))[None]
    dX = np.concatenate((np.transpose(r), P-Pd))

# iterate while any error element is greater than its tolerance
    while (np.absolute(dX) > tol).any():
	# stop execution if the maximum number of iterations is exceeded
        if iternum < Nmax:
		# compute the forward kinematics
            H = rox.fwdkin(robot, q[:,iternum])
            R = H.R
            p0T = H.p
            p0T = np.array([[p0T[0]], [p0T[1]], [p0T[2]]])
		# compute the error
            dR = np.matmul(R , np.transpose(Rd))
            r = np.array(rox.R2rpy(dR))[None]
            dX = np.concatenate((np.transpose(r), p0T-Pd))

		# calculate the Jacobian matrix
            Jq = rox.robotjacobian(robot, q[:, iternum])
		# compute the update
            j = np.matmul(np.linalg.pinv(Jq), dX)
		# use the update to generate a new q
            q[:, iternum+1] = q[:, iternum] - np.transpose((alpha * j))
            iternum = iternum + 1
        else:
            break
	# return the final estimate of q
    return q[:, iternum]

def main():

	#define inputs
	
    #qd=[30;45;80;25;40]
    #Rd = np.array([ [-0.75, -0.1047, -0.6531], [-0.433, 0.8241, 0.3652], [0.5, 0.5567, -0.6634]])
    #Pd = np.array([[0.2058], [0.1188], [0.1464]])
    #q0 = np.array(np.transpose([25, 50, 75, 30, 30]))*math.pi/180

    #qd=[0;45;135;45;135]
    Rd = np.array([ [-0.7071, -0.5, -0.5], [0, -0.7071, 0.7071], [-0.7071, 0.5, 0.5]])
    Pd = np.array([[0.1494], [0], [0.3367]])
	# convert initial guess to radians
    q0 = np.array(np.transpose([-5, 50, 130, 50, 130]))*math.pi/180
    
    tol = np.array(np.transpose([[0.02, 0.02, 0.02, 0.001, 0.001, 0.001]]))
    Nmax = 200
    alpha = 0.1

    # Define all the joint lengths [m]
    l0 = 61 * 10**-3
    l1 = 43.5 * 10**-3
    l2 = 82.85 * 10**-3
    l3 = 82.85 * 10**-3
    l4 = 73.85 * 10**-3
    l5 = 54.57 * 10**-3

	# define the unit vectors
    ex = np.array([1, 0, 0])
    ey = np.array([0, 1, 0])
    ez = np.array([0, 0, 1])

    # Define the position vectors from i-1 -> i
    P01 = (l0 + l1) * ez
    P12 = np.zeros(3)
    P23 = l2 * ex
    P34 = -1*l3 * ez
    P45 = np.zeros(3)
    P5T = -1*(l4 + l5) * ex

	# define the class inputs: rotation axes (H), position vectors (P), and joint_type
    H = np.array([ez, -1*ey, -1*ey, -1*ey, -1*ex]).T
    P = np.array([P01, P12, P23, P34, P45, P5T]).T
    joint_type = [0,0,0,0,0]

	# define the Robot class
    robot = rox.Robot(H, P, joint_type)
	
	# compute the inverse kinematics
    q = jacobian_inverse(robot,q0,Rd,Pd,Nmax,alpha,tol)
	# convert solution to degrees
    q = q * 180 / math.pi
    print(q) #this gives an array of q values for the end effector. Could be simply inputted to servo commands like we used for fk. Arm.Arm_serial_servo_write6(angle, angle, angle, angle, angle, angle-90, s_time) #FIXME
    Arm.Arm_serial_servo_write6(q[0], q[1], q[2], q[3], q[4], 90, s_time) #FIXME
    
if __name__ == "__main__" :
    main()
    
print("passed IK")


[ -0.28022737  45.25692223 134.83454941  45.17431152 134.67077778]
passed IK


In [4]:
################################################################################################################################################################################################################################################################
#PATH FOLLOWING TASK
################################################################################################################################################################################################################################################################
#IMPLEMENTED FUNCTIONS
# import numpy as np
# Other imports remain the same as your provided code

# Assuming Arm_Lib is the module that communicates with your robotic arm
from Arm_Lib import Arm_Device
import numpy as np
import general_robotics_toolbox as rox #ik
import time
import math
from Arm_Lib import Arm_Device #

# Step 1: Generate a joint space path
def generate_joint_space_path(start_angles, end_angles, steps):
    path = [np.linspace(start, end, steps) for start, end in zip(start_angles, end_angles)]
    return np.transpose(path)

# Step 2: Command the arm to follow the path
def command_arm_to_follow_path(robot, path, s_time):
    for joint_angles in path:
        # Convert angles from radians to degrees for the robot's actuators
        angles_in_degrees = joint_angles * (180 / np.pi)
        # Assuming Arm_Device is instantiated and the function Arm_serial_servo_write6 is available
        # to send the joint angles to the robot. Replace 'Arm_Device' with your actual object.
        Arm.Arm_serial_servo_write6(angles_in_degrees[0], angles_in_degrees[1], angles_in_degrees[2], angles_in_degrees[3], angles_in_degrees[4], 90, 1500)
        time.sleep(0.1)  # Wait for some time interval before sending the next set of angles

# Step 3: Measure the actual position (this function should be replaced with actual sensor readout)
def measure_actual_position(robot):
    joint_angles = []
    for i in range(1, 6):
        if Arm.Arm_serial_servo_read(i) != None:
            print(type(Arm.Arm_serial_servo_read(i)))
            joint_angles.append(Arm.Arm_serial_servo_read(i))
        else:
            print("Error None")
            joint_angles.append(0)
            
    print(joint_angles)
    # Convert the servo positions to angles, if necessary
    # joint_angles = [convert_position_to_angle(pos) for pos in joint_angles]
    time.sleep(0.1)
    return np.array(joint_angles)

# Step 4: Evaluate the error
def evaluate_path_following_error(robot, desired_path):
    error_path = []
    for desired_joint_angles in desired_path:
        actual_joint_angles = measure_actual_position(robot)
        error = np.array(desired_joint_angles) - np.array(actual_joint_angles)
        error_path.append(error)
    return error_path

def main_path():
 
    s_time = 1500 #milliseconds
    ex = np.array([1, 0, 0])
    ey = np.array([0, 1, 0])
    ez = np.array([0, 0, 1])
    #qd=[0;45;135;45;135]
    Rd = np.array([ [-0.7071, -0.5, -0.5], [0, -0.7071, 0.7071], [-0.7071, 0.5, 0.5]])
#     Rd = np.array([ [1, 0, 0], [0, 1, 0], [0, 0, 1]])

    Pd = np.array([[-0.21127], [0], [0.18735]])
	# convert initial guess to radians
#     q0 = np.array(np.transpose([-5, 50, 130, 50, 130]))*math.pi/180
    q0 = np.array(np.transpose([0, 90, 90, 90, 90]))*math.pi/180

    
    tol = np.array(np.transpose([[0.02, 0.02, 0.02, 0.001, 0.001, 0.001]]))
    Nmax = 200
    alpha = 0.1

    # Define all the joint lengths [m]
    l0 = 61 * 10**-3
    l1 = 43.5 * 10**-3
    l2 = 82.85 * 10**-3
    l3 = 82.85 * 10**-3
    l4 = 73.85 * 10**-3
    l5 = 54.57 * 10**-3
    # Define the position vectors from i-1 -> i
    P01 = (l0 + l1) * ez
    P12 = np.zeros(3)
    P23 = l2 * ex
    P34 = -1*l3 * ez
    P45 = np.zeros(3)
    P5T = -1*(l4 + l5) * ex

    H = np.array([ez, -1*ey, -1*ey, -1*ey, -1*ex]).T
    P = np.array([P01, P12, P23, P34, P45, P5T]).T
    joint_type = [0,0,0,0,0]
    robot = rox.Robot(H, P, joint_type)
    # Define your start and end angles here (in radians)
    start_angles = np.array([0, 90, 90, 90, 90]) * np.pi / 180
    end_angles = np.array([0, 60, 45, 120, 90]) * np.pi / 180
    steps = 3  # Define the number of steps you want in the path

    # Generate the path
    joint_space_path = generate_joint_space_path(start_angles, end_angles, steps)
    
    # Command the arm to follow the path
    command_arm_to_follow_path(robot, joint_space_path, s_time)

    # Measure the error
    path_following_error = evaluate_path_following_error(robot, joint_space_path)

    # Print or process the error as needed
    print("Path following errors:")
    print(path_following_error)

if __name__ == "__main__":
    main_path()
    
print("passed Path Following")

<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
[64, 85, 92, 88, 94]
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
[53, 81, 86, 92, 94]
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
[43, 78, 80, 96, 94]
Path following errors:
[array([-64.        , -83.42920367, -90.42920367, -86.42920367,
       -92.42920367]), array([-53.        , -79.69100306, -84.82190275, -90.16740429,
       -92.42920367]), array([-43.        , -76.95280245, -79.21460184, -93.9056049 ,
       -92.42920367])]
passed Path Following


In [5]:
################################################################################################################################################################################################################################################################
#FEEDBACK TASK
################################################################################################################################################################################################################################################################

In [6]:
# # Example usage of the IK function with joint limits
# target_pose = [0.2, 0.0, 0.1, 0, np.pi/2, 0]  # 
# joint_angles = calculate_ik(target_pose, joint_limits)
# print(f"The calculated joint angles for the target pose are: {joint_angles}")

In [23]:
# # Example usage
# # Define start and end joint angles and the number of steps in the path
# start_angles = np.radians([0, -np.pi/4, np.pi/2, -np.pi/4, 0, np.pi/3])
# end_angles = np.radians([np.pi/4, 0, -np.pi/2, np.pi/4, -np.pi/2, 0])
# steps = 100

# # Generate the path
# path = generate_joint_space_path(start_angles, end_angles, steps)

# # Command the arm to follow the path
# command_arm_to_follow_path(robot, path)

# # Evaluate the error
# error_path = evaluate_path_following_error(path, robot)

In [113]:
################################################################################################################################################################################################################################################################
#FK Function 1
################################################################################################################################################################################################################################################################
s_time = 1500 #milliseconds
home()
time.sleep(2) 
#from home position to blue square where cups are stacked, 35 cups
def home_to_cupstack():
    Arm.Arm_serial_servo_write6(angle-45, angle+60, angle-150, angle+50, angle, angle, s_time)
    time.sleep(1)
    Arm.Arm_serial_servo_write6(angle-45, angle+60, angle-150, angle+5, angle, angle, s_time)
    time.sleep(1)
    Arm.Arm_serial_servo_write6(angle-45, angle+60, angle-150, angle+5, angle, angle+57, s_time) 
    time.sleep(1)
    Arm.Arm_serial_servo_write6(angle-45, angle+60, angle-150, angle+5, angle, angle+57, s_time) 

home_to_cupstack()

In [114]:
################################################################################################################################################################################################################################################################
#FK Function 2
################################################################################################################################################################################################################################################################

s_time = 1500 #milliseconds

def cupstack_to_dispenser():
    Arm.Arm_serial_servo_write6(angle-45, angle+60, angle-150, angle+5, angle, angle+57, s_time) 
    time.sleep(1.5)
    Arm.Arm_serial_servo_write6(angle-45, angle+45, angle-150, angle+45, angle, angle+57, s_time) 
    time.sleep(1.5)
    Arm.Arm_serial_servo_write6(angle-26, angle+45, angle-150, angle+45, angle+180, angle+57, s_time) 
    time.sleep(1.5)
    Arm.Arm_serial_servo_write6(angle-26, angle-90, angle-70, angle+80, angle+180, angle+57, s_time) 
    time.sleep(1.5)
    Arm.Arm_serial_servo_write6(angle-26, angle-90, angle-70, angle+80, angle+180, angle, 25) 

cupstack_to_dispenser()

In [20]:
def main():
#     home() #add inherit delay to function 

try :
    main()
except KeyboardInterrupt:
     print(" Program closed! ")
     pass

IndentationError: expected an indented block (<ipython-input-20-00cae6b698dc>, line 4)

In [ ]:
Arm_Device